<div class="alert alert-info">
   <center>
       <h3>Time Series Multi-Horizon Forecasting (Dashboard) : comment prédire des phénomènes non stationnaires à des horizons de temps multiples ?</h3>
       <br>
      <p>Bonjour et bienvenue à cet atelier datacraft en collaboration avec Danone.</p>
       <p>Vous avez ici un dashboard pour tester le forecasting avec un plot via dash et plotly</p>
</center>

In [ ]:
import nbformat
import import_ipynb
import datacraft_danone as dd
import pandas as pd
import numpy as np

from jupyter_dash import JupyterDash
import dash
from dash import dcc
import dash_html_components as html
from dash.dependencies import State,Output, Input

from dash import html
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate

In [ ]:
df = pd.read_parquet("./data/forecasting.parquet.gzip")

In [ ]:
params = {
    "changepoints" : None, #lorsqu'on donne ce paramètre, le model ne va detecter aucun changepoints (ce n'est pas le paramètre par défaut)
    "n_changepoints" : 25, # nombre de changepoints # pas utile si on spécifie changepoints
    "changepoints_range" : 0.8, # n_changepoints répartis sur 80% sur train set 
    "trend_reg" : 0,
    "trend_reg_threshold" : False,
    "yearly_seasonality" :"auto",
    "weekly_seasonality" :"auto",
    "daily_seasonality": "auto",
    "seasonality_mode" : "additive",
    "seasonality_reg" : 0,
    "n_forecasts" : 1,
    "n_lags" : 0,
    "num_hidden_layers" : 0,
    "d_hidden" : None,
    "ar_reg" : None,
    "learning_rate" : None,
    "epochs" : None,
    "batch_size" : None,
    "loss_func" : "Huber",
    "optimizer" : "AdamW",
    "newer_samples_weight" : 2,
    "newer_samples_start" : 0.0,
    "impute_missing" : True,
    "collect_metrics" :True,
    "normalize" :"auto",
    "global_normalization" :False,
    "global_time_normalization" : True,
    "unknown_data_normalization" : False,
}

selected_columns = {"time_index" : 1,"apparenttemperaturemax" :1,"cos_iso_month" : 0,"cos_iso_week" : 0,"cos_iso_week_of_month": 0,"days_before_next_holiday": 0,"forecasted_volumes": 0,"fu_cod": 0,"future_ordered_volumes_until_saturday_of_current_week": 0,"future_ordered_volumes_until_saturday_of_previous_week": 0,"holiday_day_of_week": 0,"holidays_count_in_week": 0,"iso_month": 0,"iso_week": 0,"iso_week_of_month": 0,"mat_net_weight_value_kg": 0,"ordered_volumes": 1,"precipintensity": 1,"promo_uplift_coefficient": 0}

Ci-dessous, voici une fonction permettant la génération du dashboard via **dash et plotly**.

Celui-ci utilise **neural prophet** mais vous êtes libres de le modifier et d'utiliser **prophet** normal (même si le mode neural est plus performant).

Vous pourrez afficher les courbes des prédictions avec les observations réelles pour différents produits, à différentes dates avec des périodes de prédictions plus ou moins longues au choix.

Néanmoins, il faut prendre en compte un petit détail, certains produits n'ont pas des données complètes (certains ne commencent pas en 2017, mais, parfois des années après).
Il est donc possible de rencontrer des erreurs si l'année choisie est trop ancienne et donc qu'il n'y ait pas de données disponibles à cette date.


In [ ]:
def interactive_dashboard(danone, params):
    
    app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])
    app.layout = html.Div([
        html.H1('Analyse TS Danone avec Prophet'),

        dcc.Dropdown(id='dd_products', options=[{'value':k, 'label': k} for k in danone['product_index'].unique()], multi=True, value=[]),

        dcc.Dropdown(id='dd_year', options=[{'value': k, 'label': k} for k in np.sort(danone['time_index'].apply(lambda date:date.year).unique())]),

        dcc.RadioItems(id='radio_holiday', options=[{'label': 'add holidays effect', 'value': True}, {'label': "don't add holidays effect", 'value': False}]),
        
        dcc.Dropdown(id="dd_periodes", options=[{'value' : k, 'label': str(k) + "semaines"} for k in range(1,53)], multi=True, value=[]),

        dcc.Dropdown(id='dd_reg', options=[{'label': reg, 'value': reg} for reg in danone.columns], multi=True, value=[]),

        dbc.Button("apply changements", id='button'),

        dcc.Graph(id='graph_1')
    ])

    @app.callback(Output('graph_1', 'figure'),
                  Input('button', 'n_clicks'),
                  State('dd_products', 'value'),
                  State('dd_year', 'value'),
                  State('radio_holiday', 'value'),
                  State('dd_periodes', 'value'),
                  State('dd_reg', 'value')
                 )

    def plot_graph1(n_clicks, products, year, holiday_bool, periodes, reg):
        if not n_clicks:
            raise PreventUpdate
        else:
            forecasts = []
            for i in range(len(products)):
                for k in range(len(periodes)):
                    if dd.split_df(dd.prepare_df(danone, products[i], selected_columns)[0],periodes[k],year)[0].empty: 
                        return dash.no_update,'Product {} for year {} is not available, please try another year'.format(products[i], year) 
                    res = dd.train_neural_prophet_model(df = danone,
                                                        params = params,
                                                        product_id = products[i],
                                                        periode = periodes[k],
                                                        date_sep = year)
                    forecasts.append(res[3])
            print(forecasts)

            return dd.plot_my_neural_graph(forecasts = forecasts)

    app.run_server(mode='inline')

In [ ]:
interactive_dashboard(df,params)

In [ ]:
dd.plot_residuals(dd.train_prophet_model(df)[4])

In [ ]:
res = dd.train_prophet_model(df)
dd.plot_prophet_made_graph(res[3],res[0])

In [ ]:
m, danone, future, forecast, residuals = dd.train_prophet_model(df)